In [1]:
import os
import json
import pytz
from datetime import datetime, timedelta
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

class GoogleCalendarManager:
    def __init__(self):
        self.SCOPES = ['https://www.googleapis.com/auth/calendar']
        self.CALENDAR_ID = os.getenv('CALENDAR_ID')
        self.CREDENTIALS_FILE = os.getenv('CREDENTIALS_FILE')
        self.TOKEN_FILE = os.getenv('TOKEN_FILE')
        self.KOREA_TZ = pytz.timezone('Asia/Seoul')  # 한국 시간대
        self.service = self.authenticate()

    def authenticate(self):
        """구글 캘린더 API 인증"""
        creds = None
        
        # 기존 토큰 파일이 있으면 로드
        if os.path.exists(self.TOKEN_FILE):
            try:
                creds = Credentials.from_authorized_user_file(self.TOKEN_FILE, self.SCOPES)
            except (json.JSONDecodeError, ValueError):
                print("토큰 파일이 손상되어 다시 인증합니다.")
                if os.path.exists(self.TOKEN_FILE):
                    os.remove(self.TOKEN_FILE)
                creds = None
        
        # 유효하지 않은 토큰이면 새로 인증
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    self.CREDENTIALS_FILE, self.SCOPES)
                creds = flow.run_local_server(port=0)
            
            # 토큰 저장
            with open(self.TOKEN_FILE, 'w') as token:
                token.write(creds.to_json())
        
        return build('calendar', 'v3', credentials=creds)

    def check_duplicate_event(self, title, start_time):
        """중복 일정 확인 (한국 시간대 적용)"""
        try:
            # 시작 시간이 naive datetime이면 한국 시간대 정보 추가
            if start_time.tzinfo is None:
                start_time_kst = self.KOREA_TZ.localize(start_time)
            else:
                start_time_kst = start_time.astimezone(self.KOREA_TZ)
            
            # 하루 전후로 검색 범위 설정 (한국 시간대)
            time_min = (start_time_kst - timedelta(days=1)).isoformat()
            time_max = (start_time_kst + timedelta(days=1)).isoformat()
            
            events_result = self.service.events().list(
                calendarId=self.CALENDAR_ID,
                timeMin=time_min,
                timeMax=time_max,
                singleEvents=True,
                orderBy='startTime'
            ).execute()
            
            events = events_result.get('items', [])
            
            # 같은 제목과 시작 시간인 일정이 있는지 확인
            for event in events:
                if event.get('summary') == title:
                    event_start = event['start'].get('dateTime', event['start'].get('date'))
                    if event_start.startswith(start_time_kst.strftime('%Y-%m-%d')):
                        return True
            return False
            
        except HttpError as error:
            print(f'⚠️  중복 확인을 건너뛰고 일정을 생성합니다. (오류: {error})')
            return False

    def create_event(self, title, start_time, end_time, location='', description='', attendees=None):
        """일정 생성 (한국 시간대 적용)"""
        # 중복 확인
        if self.check_duplicate_event(title, start_time):
            print(f"⚠️  '{title}' 일정이 이미 존재합니다.")
            return None
        
        # 시간대 정보 추가
        if start_time.tzinfo is None:
            start_time = self.KOREA_TZ.localize(start_time)
        if end_time.tzinfo is None:
            end_time = self.KOREA_TZ.localize(end_time)
        
        event = {
            'summary': title,
            'location': location,
            'description': description,
            'start': {
                'dateTime': start_time.isoformat(),
                'timeZone': 'Asia/Seoul',
            },
            'end': {
                'dateTime': end_time.isoformat(),
                'timeZone': 'Asia/Seoul',
            },
        }
        
        # 참석자 추가
        if attendees:
            event['attendees'] = [{'email': email} for email in attendees]
        
        try:
            event = self.service.events().insert(
                calendarId=self.CALENDAR_ID, 
                body=event
            ).execute()
            print(f'✅ 일정이 생성되었습니다 (한국시간): {event.get("htmlLink")}')
            return event
        except HttpError as error:
            print(f'일정 생성 중 오류 발생: {error}')
            return None

    def list_events(self, max_results=10):
        """일정 목록 조회 (한국 시간대)"""
        try:
            # 현재 한국 시간 가져오기
            now_kst = datetime.now(self.KOREA_TZ)
            now_iso = now_kst.isoformat()
            
            events_result = self.service.events().list(
                calendarId=self.CALENDAR_ID,
                timeMin=now_iso,
                maxResults=max_results,
                singleEvents=True,
                orderBy='startTime'
            ).execute()
            
            events = events_result.get('items', [])
            
            if not events:
                print('다가오는 일정이 없습니다.')
                return []
            
            print(f'\n📅 다가오는 {len(events)}개 일정 (한국시간):')
            for i, event in enumerate(events, 1):
                start = event['start'].get('dateTime', event['start'].get('date'))
                # 시간 표시를 한국 시간으로 변환
                if 'T' in start:  # dateTime 형식인 경우
                    try:
                        dt = datetime.fromisoformat(start.replace('Z', '+00:00'))
                        dt_kst = dt.astimezone(self.KOREA_TZ)
                        start_display = dt_kst.strftime('%Y-%m-%d %H:%M (KST)')
                    except:
                        start_display = start
                else:  # date 형식인 경우
                    start_display = start
                
                print(f"{i}. {event['summary']} - {start_display}")
                if event.get('location'):
                    print(f"    📍 장소: {event['location']}")
                if event.get('description'):
                    print(f"    📝 설명: {event['description']}")
            
            return events
        except HttpError as error:
            print(f'일정 조회 중 오류 발생: {error}')
            return []

    def get_event_by_id(self, event_id):
        """특정 일정 정보 조회"""
        try:
            event = self.service.events().get(
                calendarId=self.CALENDAR_ID, 
                eventId=event_id
            ).execute()
            return event
        except HttpError as error:
            print(f'일정 정보 조회 중 오류 발생: {error}')
            return None

    def update_event(self, event_id, **kwargs):
        """일정 수정 (한국 시간대 적용)"""
        try:
            # 기존 일정 정보 가져오기
            event = self.get_event_by_id(event_id)
            if not event:
                return None
            
            # 수정할 내용 업데이트
            for key, value in kwargs.items():
                if key == 'title':
                    event['summary'] = value
                elif key == 'start_time':
                    if value.tzinfo is None:
                        value = self.KOREA_TZ.localize(value)
                    event['start']['dateTime'] = value.isoformat()
                elif key == 'end_time':
                    if value.tzinfo is None:
                        value = self.KOREA_TZ.localize(value)
                    event['end']['dateTime'] = value.isoformat()
                elif key == 'location':
                    event['location'] = value
                elif key == 'description':
                    event['description'] = value
                elif key == 'attendees':
                    event['attendees'] = [{'email': email} for email in value] if value else []
            
            updated_event = self.service.events().update(
                calendarId=self.CALENDAR_ID,
                eventId=event_id,
                body=event
            ).execute()
            
            print(f'✅ 일정이 수정되었습니다: {updated_event.get("summary")}')
            return updated_event
        except HttpError as error:
            print(f'일정 수정 중 오류 발생: {error}')
            return None

    def delete_event(self, event_id):
        """일정 삭제"""
        try:
            # 삭제하기 전에 일정 정보 확인
            event = self.get_event_by_id(event_id)
            if event:
                event_title = event.get('summary', '제목 없음')
                
                self.service.events().delete(
                    calendarId=self.CALENDAR_ID,
                    eventId=event_id
                ).execute()
                print(f'✅ 일정 "{event_title}"이(가) 삭제되었습니다.')
                return True
            else:
                print('일정을 찾을 수 없습니다.')
                return False
        except HttpError as error:
            print(f'일정 삭제 중 오류 발생: {error}')
            return False


def get_user_input():
    """사용자로부터 일정 정보 입력받기"""
    print("\n" + "="*50)
    print("📝 새 일정 정보를 입력하세요")
    print("="*50)
    
    title = input("일정 제목: ").strip()
    if not title:
        print("❌ 일정 제목은 필수입니다.")
        return None
    
    # 시작 날짜 및 시간 입력
    while True:
        try:
            start_date = input("시작 날짜 (YYYY-MM-DD): ").strip()
            start_time = input("시작 시간 (HH:MM): ").strip()
            start_datetime = datetime.strptime(f"{start_date} {start_time}", "%Y-%m-%d %H:%M")
            break
        except ValueError:
            print("❌ 날짜/시간 형식이 올바르지 않습니다. 다시 입력해주세요.")
    
    # 종료 시간 입력
    while True:
        try:
            duration_input = input("일정 시간 (시간 단위, 기본값: 1): ").strip()
            duration = int(duration_input) if duration_input else 1
            end_datetime = start_datetime + timedelta(hours=duration)
            break
        except ValueError:
            print("❌ 숫자만 입력해주세요.")
    
    location = input("장소 (선택사항): ").strip()
    description = input("설명 (선택사항): ").strip()
    
    attendees_input = input("참석자 이메일 (쉼표로 구분, 선택사항): ").strip()
    attendees = [email.strip() for email in attendees_input.split(',') if email.strip()] if attendees_input else None
    
    return title, start_datetime, end_datetime, location, description, attendees


def get_update_input():
    """일정 수정 정보 입력받기"""
    print("\n수정할 내용을 입력하세요 (엔터만 누르면 기존값 유지):")
    
    update_data = {}
    
    new_title = input("새 제목: ").strip()
    if new_title:
        update_data['title'] = new_title
    
    new_location = input("새 장소: ").strip()
    if new_location:
        update_data['location'] = new_location
    
    new_description = input("새 설명: ").strip()
    if new_description:
        update_data['description'] = new_description
    
    # 시간 수정은 복잡하므로 선택사항으로 처리
    change_time = input("시간도 수정하시겠습니까? (y/N): ").strip().lower()
    if change_time == 'y':
        try:
            new_date = input("새 시작 날짜 (YYYY-MM-DD): ").strip()
            new_time = input("새 시작 시간 (HH:MM): ").strip()
            new_start = datetime.strptime(f"{new_date} {new_time}", "%Y-%m-%d %H:%M")
            
            duration_input = input("새 일정 시간 (시간 단위): ").strip()
            duration = int(duration_input) if duration_input else 1
            new_end = new_start + timedelta(hours=duration)
            
            update_data['start_time'] = new_start
            update_data['end_time'] = new_end
        except ValueError:
            print("❌ 시간 형식이 올바르지 않아 시간 수정을 건너뜁니다.")
    
    return update_data if update_data else None


def main():
    """메인 프로그램"""
    try:
        calendar = GoogleCalendarManager()
        print("🎉 구글 캘린더 API 연결 성공!")
    except FileNotFoundError:
        print("❌ credentials.json 파일을 찾을 수 없습니다.")
        print("구글 클라우드 콘솔에서 OAuth 2.0 클라이언트 ID를 생성하고 credentials.json 파일을 다운로드하세요.")
        return
    except Exception as e:
        print(f"❌ 초기화 오류: {e}")
        return
    
    while True:
        print("\n" + "="*60)
        print("📅 구글 캘린더 관리 프로그램 (한국시간)")
        print("="*60)
        print("1. 📝 일정 추가")
        print("2. 📋 일정 목록 보기") 
        print("3. ✏️  일정 수정")
        print("4. 🗑️  일정 삭제")
        print("5. 🔍 일정 상세 조회")
        print("6. 🚪 종료")
        
        choice = input("\n선택하세요 (1-6): ").strip()
        
        if choice == '1':
            # 일정 추가
            result = get_user_input()
            if result:
                title, start_time, end_time, location, description, attendees = result
                calendar.create_event(title, start_time, end_time, location, description, attendees)
        
        elif choice == '2':
            # 일정 목록 보기
            max_results_input = input("표시할 일정 개수 (기본값: 10): ").strip()
            max_results = int(max_results_input) if max_results_input.isdigit() else 10
            calendar.list_events(max_results)
        
        elif choice == '3':
            # 일정 수정
            print("\n✏️  일정 수정")
            events = calendar.list_events()
            if events:
                try:
                    index = int(input("\n수정할 일정 번호: ").strip()) - 1
                    if 0 <= index < len(events):
                        event_id = events[index]['id']
                        update_data = get_update_input()
                        if update_data:
                            calendar.update_event(event_id, **update_data)
                        else:
                            print("수정할 내용이 없습니다.")
                    else:
                        print("❌ 올바르지 않은 번호입니다.")
                except ValueError:
                    print("❌ 숫자를 입력해주세요.")
        
        elif choice == '4':
            # 일정 삭제
            print("\n🗑️  일정 삭제")
            events = calendar.list_events()
            if events:
                try:
                    index = int(input("\n삭제할 일정 번호: ").strip()) - 1
                    if 0 <= index < len(events):
                        event_id = events[index]['id']
                        event_title = events[index]['summary']
                        
                        confirm = input(f"'{event_title}' 일정을 정말 삭제하시겠습니까? (y/N): ").strip().lower()
                        if confirm == 'y':
                            calendar.delete_event(event_id)
                        else:
                            print("삭제가 취소되었습니다.")
                    else:
                        print("❌ 올바르지 않은 번호입니다.")
                except ValueError:
                    print("❌ 숫자를 입력해주세요.")
        
        elif choice == '5':
            # 일정 상세 조회
            print("\n🔍 일정 상세 조회")
            events = calendar.list_events()
            if events:
                try:
                    index = int(input("\n조회할 일정 번호: ").strip()) - 1
                    if 0 <= index < len(events):
                        event = events[index]
                        print("\n" + "="*50)
                        print(f"📅 일정 상세 정보")
                        print("="*50)
                        print(f"제목: {event.get('summary', '제목 없음')}")
                        
                        # 시간 정보
                        start = event['start'].get('dateTime', event['start'].get('date'))
                        end = event['end'].get('dateTime', event['end'].get('date'))
                        print(f"시작: {start}")
                        print(f"종료: {end}")
                        
                        # 추가 정보
                        if event.get('location'):
                            print(f"장소: {event['location']}")
                        if event.get('description'):
                            print(f"설명: {event['description']}")
                        if event.get('attendees'):
                            attendees = [attendee['email'] for attendee in event['attendees']]
                            print(f"참석자: {', '.join(attendees)}")
                        
                        print(f"링크: {event.get('htmlLink', '없음')}")
                    else:
                        print("❌ 올바르지 않은 번호입니다.")
                except ValueError:
                    print("❌ 숫자를 입력해주세요.")
        
        elif choice == '6':
            print("👋 프로그램을 종료합니다.")
            break
        
        else:
            print("❌ 올바르지 않은 선택입니다. 1-6 중에서 선택해주세요.")


if __name__ == '__main__':
    main()


🎉 구글 캘린더 API 연결 성공!

📅 구글 캘린더 관리 프로그램 (한국시간)
1. 📝 일정 추가
2. 📋 일정 목록 보기
3. ✏️  일정 수정
4. 🗑️  일정 삭제
5. 🔍 일정 상세 조회
6. 🚪 종료

📝 새 일정 정보를 입력하세요
✅ 일정이 생성되었습니다 (한국시간): https://www.google.com/calendar/event?eid=bnRuM2s2azgxYzRxYXZpNGEybzNsN2lraDAgNDZlMjFkYzhhMjhlZmI0ODg4YmY5NTJmZjg4Y2QxNTE0Y2Y1ZGJlYTlmYWViMTU2MTViNzVjMTM5MWNjMmJjMUBn

📅 구글 캘린더 관리 프로그램 (한국시간)
1. 📝 일정 추가
2. 📋 일정 목록 보기
3. ✏️  일정 수정
4. 🗑️  일정 삭제
5. 🔍 일정 상세 조회
6. 🚪 종료
👋 프로그램을 종료합니다.
